In [ ]:
#dates in the index


"""  
Dates in the index

"""
import pandas as pd


#url = 'https://raw.githubusercontent.com/mattharrison/datasets/master/data/alta-noaa-1980-2019.csv'

#alta_df =  pd.read_csv(url)


# had to install pyarrow
#alta_df.to_parquet('alta_df.parquet.gzip', compression='gzip')


alta_df = pd.read_parquet('alta_df.parquet.gzip')


dates = pd.to_datetime(alta_df.DATE)

snow = (alta_df
        .SNOW
        .rename(dates))

In [ ]:
snow

In [ ]:
snow.isna().any()

In [ ]:
snow[snow.isna()]


In [ ]:
snow.loc['1985-09':'1985-09-20']

In [ ]:
(snow
 .loc['1985-09':'1985-09-20']
 .fillna(0))

In [ ]:
snow.loc['1987-12-30':'1988-01-10']

In [ ]:
(snow
 .loc['1987-12-30':'1988-01-10']
 .ffill())

In [ ]:
(snow
 .loc['1987-12-30':'1988-01-10']
 .bfill())

In [ ]:
(snow
 .loc['1987-12-30':'1988-01-10'])

In [ ]:
(snow
 .loc['1987-12-30':'1988-01-10']
 .interpolate())

In [ ]:
winter = (snow.index.quarter == 1) | (snow.index.quarter ==4)

(snow
 .where(~(winter & snow.isna()), snow.interpolate())
 .where(~(~winter & snow.isna()), 0))

In [ ]:
(snow
 .where(~(winter & snow.isna()), snow.interpolate())
 .where(~(~winter & snow.isna()), 0)
 .loc[['1985-09-19','1988-01-01']]
 )

In [ ]:
# drop the missing data

(snow
 .loc['1987-12-30':'1988-01-10']
 .dropna())

In [ ]:
snow.shift(1)

In [ ]:
snow.shift(-1)

In [ ]:
(snow
 .add(snow.shift(1))
 .add(snow.shift(2))
 .add(snow.shift(3))
 .add(snow.shift(4))
 .div(5)
 )

In [33]:
(snow
 .rolling(5)
 .mean()
 )

1980-01-01    NaN
1980-01-02    NaN
1980-01-03    NaN
1980-01-04    NaN
1980-01-05    1.2
             ... 
2019-09-03    0.0
2019-09-04    0.0
2019-09-05    0.0
2019-09-06    0.0
2019-09-07    0.0
Name: SNOW, Length: 14160, dtype: float64

13.7 Resampling

In [34]:


(snow
 .resample('M')
 .max()
)




1980-01-31    20.0
1980-02-29    25.0
1980-03-31    16.0
1980-04-30    10.0
1980-05-31     9.0
              ... 
2019-05-31     5.1
2019-06-30     0.0
2019-07-31     0.0
2019-08-31     0.0
2019-09-30     0.0
Freq: M, Name: SNOW, Length: 477, dtype: float64

In [35]:
(snow
 .resample('2M')
 .max())

1980-01-31    20.0
1980-03-31    25.0
1980-05-31    10.0
1980-07-31     1.0
1980-09-30     0.0
              ... 
2019-01-31    19.0
2019-03-31    20.7
2019-05-31    18.0
2019-07-31     0.0
2019-09-30     0.0
Freq: 2M, Name: SNOW, Length: 239, dtype: float64

In [ ]:
(snow
 .resample('A-MAY')
 .max()
 )

13.8  Gathering Aggregate Values (But Keeping Index  )

In [ ]:
(snow
 .div(snow
      .resample('Q')
      .transform('sum'))
 .mul(100)
 .fillna(0)    
)

In [44]:
(snow['1990-01':'1990-01']
 
)

1990-01-01     0.0
1990-01-02    17.0
1990-01-03     4.0
1990-01-04     0.0
1990-01-05    13.0
1990-01-06     0.0
1990-01-07     0.0
1990-01-08     9.0
1990-01-09     2.0
1990-01-10     0.0
1990-01-11     0.0
1990-01-12     0.0
1990-01-13     5.0
1990-01-14     9.0
1990-01-15     2.0
1990-01-16     5.0
1990-01-17     0.0
1990-01-18     0.0
1990-01-19     0.0
1990-01-20     0.0
1990-01-21     0.0
1990-01-22     0.0
1990-01-23     0.0
1990-01-24     6.0
1990-01-25     0.0
1990-01-26     4.0
1990-01-27     NaN
1990-01-28     1.0
1990-01-29     4.0
1990-01-30     1.0
1990-01-31     9.0
Name: SNOW, dtype: float64

Figure 13.4: Note numbers are different than in the book

In [39]:
(snow['1990-01':'1990-05']
 .resample('M')
 .sum()
)

1990-01-31    91.0
1990-02-28    73.0
1990-03-31    56.0
1990-04-30    25.0
1990-05-31     4.0
Freq: M, Name: SNOW, dtype: float64

In [45]:
t2= (snow
 .resample('M')
 .transform('sum')
)

In [43]:
t = (snow
 .resample('M')
 .sum()
)

In [46]:
season2017 = snow.loc['2016-10':'2017-05']

(season2017
 .resample('M')
 .sum()
 .div(season2017.sum())
 .mul(100)
 
 )

2016-10-31     2.153969
2016-11-30     9.772637
2016-12-31    15.715995
2017-01-31    25.468688
2017-02-28    21.041085
2017-03-31     9.274033
2017-04-30    14.738732
2017-05-31     1.834862
Freq: M, Name: SNOW, dtype: float64

13.9 Groupby Operations

In [ ]:
#group by operations

def season(idx):
    year = idx.year
    month = idx.month
    return year.where((month < 10), year +1)


(snow
 .groupby(season)
 .sum()
 
 )

In [ ]:
(snow
 .resample('A-SEP')
 .sum()
 )

13.10 Cumulative Operations

In [36]:
(snow
 .loc['2016-10':'2017-09']
 .cumsum()
 )

2016-10-01      0.0
2016-10-02      0.0
2016-10-03      4.9
2016-10-04      4.9
2016-10-05      5.5
              ...  
2017-09-26    524.0
2017-09-27    524.0
2017-09-28    524.0
2017-09-29    524.0
2017-09-30    524.0
Name: SNOW, Length: 364, dtype: float64

In [47]:
(snow
    .resample('A-SEP')
    .transform('cumsum')
)

1980-01-01      2.0
1980-01-02      5.0
1980-01-03      6.0
1980-01-04      6.0
1980-01-05      6.0
              ...  
2019-09-03    504.5
2019-09-04    504.5
2019-09-05    504.5
2019-09-06    504.5
2019-09-07    504.5
Name: SNOW, Length: 14160, dtype: float64

13:12  Exercise
